In [44]:
import os
import AudioConverter
import re
import io
from playsound import playsound

## Step 1: Transform needed file into flac

In [ ]:
!ffmpeg -i Stay_Hungry_Stay_Foolish.mp3 -c:a flac Stay_Hungry_Stay_Foolish.flac ## For the imformaiton of ffmpeg, please visit
## https://ffmpeg.org/

## Step 2: Upload the flac file to Google Cloud Platform storage

## Step 3: Get Ready to get transcription of the audio from Google Cloud API

In [10]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="elegant-hope-XXX010-adXXXXXXXX80.json" #For API keys, please visit Google Cloud Platform
# https://cloud.google.com/docs/authentication/api-keys

In [11]:
from google.cloud import speech_v1p1beta1 as speech
from google.cloud.speech_v1p1beta1 import enums
from google.cloud.speech_v1p1beta1 import types

In [12]:
from pydub import AudioSegment
import wave
from google.cloud import storage

In [13]:

intervals = (
    ('weeks', 604800),  # 60 * 60 * 24 * 7
    ('days', 86400),    # 60 * 60 * 24
    ('hours', 3600),    # 60 * 60
    ('minutes', 60),
    ('seconds', 1),
    )

def display_time(seconds, granularity=2):
    result = []

    for name, count in intervals:
        value = seconds // count
        if value:
            seconds -= value * count
            if value == 1:
                name = name.rstrip('s')
            result.append("{}".format(value))
    return ':'.join(result[:granularity])

In [40]:
from google.cloud import speech_v1


def sample_long_running_recognize(storage_uri, filename, language):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognition

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """
    file1 = open(f"{filename}.txt","w", encoding="utf-8") 
    client = speech.SpeechClient()

    # storage_uri = 'gs://cloud-samples-data/speech/brooklyn_bridge.raw'
    enable_word_time_offsets = True
    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 44100  #Sometimes we need to change this by hand according to the error messages

    # The language of the supplied audio
    language_code = language

    enable_speaker_diarization = True
    diarization_speaker_count = 1
    
    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.

    encoding = enums.RecognitionConfig.AudioEncoding.FLAC
    config = {
        "sample_rate_hertz": sample_rate_hertz,
        "language_code": language_code,
        "encoding": encoding,
        "enable_word_time_offsets": enable_word_time_offsets,
        "enable_speaker_diarization": enable_speaker_diarization,
        "diarization_speaker_count": diarization_speaker_count,
        "audio_channel_count" : 2 #this config depends on how many channels your audio has.

    }
 
    #with io.open(local_file_path, "rb") as f:
     #   content = f.read()
    #audio = {"content": content}
 
    audio = {"uri": storage_uri}

    operation = client.long_running_recognize(config, audio)

    print(u"Waiting for operation to complete...")
    response = operation.result()

    for result in response.results:
        # First alternative is the most probable result

        alternative = result.alternatives[0]
        seconds = alternative.words[0].end_time.seconds
        time = display_time(seconds, granularity=3)
        print(f"AB:{alternative.transcript}\n")
        file1.write(f"AB:{alternative.transcript}\n")


    file1.close()

In [41]:
storage_uri = 'gs://toseanliu_speech_to_text/Stay_Hungry_Stay_Foolish.flac'
a = re.sub("gs://toseanliu_speech_to_text/", "", storage_uri)
filename = re.sub(".flac", "", a)
filename

'Stay_Hungry_Stay_Foolish'

In [42]:
storage_uri

'gs://toseanliu_speech_to_text/Stay_Hungry_Stay_Foolish.flac'

In [43]:
#zh-TW en-US zh-CN zh-HK
sample_long_running_recognize(storage_uri, filename, "en-US") #The language used is en-US

Waiting for operation to complete...
AB:I'm honored to be with you today for your commencement from one of the finest universities in the world

AB: truth be told

AB: I never graduated from college and this is the closest I've ever gotten to a college graduation

AB: today I want to tell you three stories from my life that's it no big deal just three stories

AB: the first story is about connecting the dots

AB: I dropped out of Reed College after the first six months but then stayed around as a drop in for another 18 months or so before I really quit

AB: so why did I drop out

AB: it started before I was born

AB: my biological mother was a young unwed graduate student and she decided to put me up for adoption she felt very strongly that I should be adopted by college graduates so everything was all set for me to be adopted at Birth by a lawyer and his wife

AB: accept it when I popped out they decided at the last minute that they really wanted a girl

AB: so my parents who are on a